# Analyse von Veröffentlichungen mit KI Bezug von CSRankings.org

## Rohdaten

Die Datensätze wurden mit dem Code unter https://github.com/emeryberger/CSrankings erzeugt.

Erstellungsdatum: 06.01.2019

In [1]:
import pandas as pd

affiliations = pd.read_csv("./data/faculty-affiliations.csv")
universities = pd.read_csv("./data/country-info.csv")
articles = pd.read_json("./data/articles.json")

## Hochschulen
Filtern Europäischer Hochschulen (durch Attribut in Datensatz)

In [0]:
european_universities = universities[universities["region"] == "europe"]

Ausgabe der europäischen Universitäten zur manuellen Filterung hessischer/deutscher Einrichtungen

In [0]:
pd.options.display.max_rows = -1 # do not abbreviate the list
european_universities

Manuelle Identifizierung Deutscher Hochschulen und Ausgabe zur Filterung Hessischer Hochschulen

In [0]:
german_universities = european_universities.loc[[13, 72, 78, 94, 95, 105, 106, 107, 109, 111, 142, 147, 159, 184, 199, 200]]
german_universities

Manuelle Identifizierung Hessischer Hochschulen

In [0]:
hessian_universities = german_universities.loc[[107,]]

## Artikel
Aussortieren doppelter Einträge (nach Titel)

In [0]:
before = len(articles)
articles = articles.drop_duplicates(subset="title")
after = len(articles)

"{} Duplikate entfernt".format(before - after)

Auswählen der Artikel aus "KI"-Konferenzen

In [0]:
ai_articles = articles[articles.area.isin(["aaai", "ijcai", "cvpr", "eccv", "iccv", "icml", "kdd", "nips", "acl", "emnlp", "naacl", "sigr", "www"])]
"KI-Artikel insgesamt: {}".format(len(ai_articles))

Matching der KI-Artikel mit den deutschen Hochschulen

In [0]:
german_ai_articles = ai_articles.join(german_universities.set_index("institution"), on="institution", how="inner")

In [0]:
"KI-Artikel von deutschen Universitäten: {}".format(len(german_ai_articles))

Einfügen einer Spalte `from_hessen` in die Tabelle der Deutschen KI-Artikel um Artikel Hessischer Hochschulen zu kennzeichen

In [0]:
german_ai_articles["from_hessen"] = german_ai_articles["institution"].isin(hessian_universities["institution"])
"KI-Artikel hessicher Universitäten: {}".format(len(german_ai_articles[german_ai_articles["from_hessen"]==True]))

### Abrufen der Anzahl an Zitierungen von Google Scholar

In [0]:
from scholar.scholar import ScholarQuerier, SearchScholarQuery

def fetch_citation_count(publication):
    querier = ScholarQuerier()
    query = SearchScholarQuery()
    
    author = publication["name"]
    title = publication["title"]
    
    print("Autor: {}, Titel: {}".format(author, title))
    
    query.set_author(author)
    query.set_words(title)
    query.set_scope(True)
    query.set_include_patents(False)
    query.set_num_page_results(1)
    
    querier.send_query(query)
    
    articles = querier.articles
    print(articles)
    

german_ai_articles.apply(fetch_citation_count, axis=1)

## Visualiserung/Export

In [3]:
german_ai_articles

NameError: name 'german_ai_articles' is not defined

Export der Liste deutsche KI-Artikel nach `publications.csv`

In [5]:
try:
    german_ai_articles.to_csv("publications.csv")
except NameError:
    german_ai_articles = pd.read_csv("publications.csv")

Säulindiagramm der Anzahl Deutscher und Hessischer Beiträge zu "KI-Konferenzen"

In [11]:
import plotly.offline as py
import plotly.graph_objs as go

vc = german_ai_articles[german_ai_articles["from_hessen"]==False]["year"].value_counts()
vc_hessen = german_ai_articles[german_ai_articles["from_hessen"]==True]["year"].value_counts()

data = [
    go.Bar(
        x=vc.index.values,
        y=vc.values,
        name = "Deutschland Rest"
    ),
    go.Bar(
        x=vc_hessen.index.values,
        y=vc_hessen.values,
        name="Hessen"
    )
]

layout = go.Layout(
    barmode='stack',
    title='Beiträge zu KI-Konferenzen Deutscher Hochschulen nach Jahren'
)

fig = go.Figure(data=data, layout=layout)
chart = py.plot(fig, filename="publications.html")
ex = pd.DataFrame()
ex['hessen'] = vc_hessen
ex['de_rest'] = vc
ex = ex.sort_index()
ex.to_csv("anzahl_an_ki_konferenzbeiträgen.csv")

![chart](publications.png)